In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")
model.eval()




ValueError: batch length of `text`: 1 does not match batch length of `text_pair`: 2.

In [3]:
references = ["hello world", "hello world"]
candidates = ["hi universe", "bye world"]

with torch.no_grad():
  scores = model(**tokenizer(references, candidates, return_tensors='pt'))[0].squeeze()

print(scores) # tensor([1.0327, 0.2055])

tensor([1.0327, 0.2055])


In [6]:
references = [f'Given that the likelihoods of class C1 and class C2 are 36.38% and 63.62%, respectively, the case under consideration is labelled as C2 by the model. The top relevant features for this prediction decision are F9, F12, F16, and F6, while the least relevant features include F5 and F1. From the analysis performed, the values of F9, F12, and F16 are shown to have strong positive support for C2, while F6 and F13 are the top negative attributes, pulling the prediction towards C1. However, the combined magnitude of their impacts on the model is less than that of the positive input features mentioned above, hence the moderately strong push towards labelling the case as C2. The values of some features show a low influence on C2 prediction and these are F2, F10, F5, and F1.']
candidates = ['The model predicted class C2 for the given case with a confidence level of 63.62%. However, there is a 36.38% chance that it could be C1 instead. The above prediction decision is mainly due to the influence of features such as F9, F6, and F12. On the other hand, the least important features are F5 and  F1. In terms of the direction of influence or contribution of each input feature, only F3 has a negative impact, shifting the prediction verdict in favour of an alternative label. Finally, on the contrary, all the remaining features have positive contributions, increasing the likelihood of labelling the provided example as "C2". All things considered, we can conclude that the uncertainty associated with the classification decision here may be attributed to some combination of negative features coupled with strong positive attributions from F11, F7, And F4.']

with torch.no_grad():
  scores = model(**tokenizer(references, candidates, return_tensors='pt'))[0].squeeze()

print(scores) # tensor([1.0327, 0.2055])

tensor(-0.0436)


In [7]:
import evaluate

In [9]:
rouge = evaluate.load('rouge')
# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=candidates,
                         references=references)
print(results)

{'rouge1': 0.5, 'rouge2': 0.11510791366906475, 'rougeL': 0.2571428571428572, 'rougeLsum': 0.2571428571428572}
